<a href="https://colab.research.google.com/github/Vazimax/comment_toxicity/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [31]:
data = pd.read_csv('/train.csv', error_bad_lines=False, engine="python")
X = data['comment_text']
y = data[data.columns[2:]].values


<ipython-input-31-c223c677d7c0>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('/train.csv', error_bad_lines=False, engine="python")
Skipping line 14358: unexpected end of data


In [32]:
max_features = 20000
vectorizer = TextVectorization(max_tokens=max_features, output_sequence_length=1500, output_mode='int')

In [33]:
vectorizer.adapt(X.values)

In [35]:
vectorized_text = vectorizer(X.values)

In [36]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache().shuffle(160000).batch(16).prefetch(tf.data.AUTOTUNE)

In [37]:
train_size = int(len(dataset) * 0.7)
val_size = int(len(dataset) * 0.2)
test_size = int(len(dataset) * 0.1)

In [38]:
train = dataset.take(train_size)
val = dataset.skip(train_size).take(val_size)
test = dataset.skip(train_size + val_size)

In [39]:
model = Sequential([
    Embedding(max_features + 1, 32),
    Bidirectional(LSTM(32, activation='tanh')),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(6, activation='sigmoid')
])

In [40]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [41]:
history = model.fit(train, epochs=1, validation_data=val)

628/628 [==============================] - 953s 2s/step - loss: 0.1197 - val_loss: 0.0773


In [43]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [44]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [45]:
for batch in test.as_numpy_iterator():
    # Unpack the batch
    X_true, y_true = batch
    # Make a prediction
    yhat = model.predict(X_true)

    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()

    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 142ms/step


In [48]:
print(f' Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

 Precision: 0.8397790193557739, Recall:0.5049833655357361, Accuracy:0.5164835453033447
